In [58]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(400)
import collections
import re
import nltk
!pip install tika
import tika
import glob
tika.initVM()
import seaborn as sns
from tika import parser
from nltk.stem import SnowballStemmer
import os
import pandas as pd
import gensim, nltk, os
nltk.download('punkt')
from nltk.corpus import stopwords
nltk.download('stopwords')
import gensim
from gensim import corpora
import pickle
import random


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/sapnasharma/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/sapnasharma/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Do as follows if guidedlda in not getting installed with pip

from command window

git clone https://github.com/vi3k6i5/GuidedLDA
cd GuidedLDA
ls
sh build_dist.sh
python setup.py sdist
pip3 install -e .

pip3 install -U cython

remove the following 2 lines from setup.cfg:

[sdist]
pre-hook.sdist_pre_hook = guidedlda._setup_hooks.sdist_pre_hook




In [118]:
'''
pre processing steps on the entire dataset
'''
# importing customized stopwords from customized_stopwords.txt
with open ('customized_stopwords', 'rb') as fp:
    customized_stopwords = pickle.load(fp)
more_stop_words = ['finish','start','tomorrow','work','agree','think','middle','dicide','write','haven','understand','print','call','return','talk','happen']   
customized_stopwords=customized_stopwords + more_stop_words
#stemmer = SnowballStemmer("english")
def lemmatize(word):                                    # input is a word that is to be converted to root word for verb
    return WordNetLemmatizer().lemmatize(word, pos = 'v')

def preprocess(text):
    result=[]
    for token in gensim.utils.simple_preprocess(text) :
        if (token not in gensim.parsing.preprocessing.STOPWORDS) and (len(token) > 4) and (token not in customized_stopwords):
            if token not in customized_stopwords:
                
                result.append(lemmatize(token))
                if token == 'happen':
                    print("yy")
            
    return result

In [120]:
# reading all documents
combined_words = ""
docs = []
for transcript_file_name in glob.iglob('./transcripts/train//*.*', recursive=True):
    print(os.path.basename(transcript_file_name))
    data = open(transcript_file_name).readlines()
    speaker_data = {line.split(":")[0]:line.split(":")[1] for line in data}
    words_in_file = ""
    speaker_dic ={}
    for name,words in  speaker_data.items():
        words = words.replace("\n","").lower()
        words_in_file = words_in_file + words
        if name.split("_")[0] in speaker_dic:
            speaker_dic[name.split("_")[0]] += words
        else:
            speaker_dic[name.split("_")[0]] = words
    #print("Number of words in the file :",str(len(words_in_file)))
    combined_words += words_in_file
    docs.append([words_in_file])
        

autosampler-Onfido_2020-07-10_transcript.txt
Speech_Diarization_2020-07-22_transcript.txt
autosampler-Onfido_2020-07-17_transcript.txt
Embedder.m4a_transcript_2.txt
Website_Optimization_2020-07-20_transcript.txt
Website_Optimization_2020-07-28_transcript.txt
Embedder.m4a_transcript_3.txt
Embedder.m4a_transcript_1.txt
Website_Optimization_2020-07-22_transcript.txt
Speech_Diarization.m4a_transcript.txt
Speech_Diarization_2020-07-28_transcript.txt
Embedder_2020-07-28_transcript.txt
Website_Optimization.m4a_transcript_Jul_29.txt
Website_Optimization.m4a_transcript_1.txt
Website_Optimization.m4a_transcript.txt
autosampler-Onfido_2020-07-22_transcript.txt
Embedder.m4a_transcript.txt
autosampler-Onfido_2020-07-23_transcript.txt
Speech_Diarization.m4a_transcript_1.txt
autosampler-Onfido.m4a_transcript.txt
Embedder_2020-07-22_transcript.txt
Speech_Diarization.m4a_transcript_2.txt


In [121]:
cleaned_docs = []
for doc in docs:
    for word in doc:
        cd = preprocess(word)
        cleaned_docs.append(cd)

In [138]:
#len(cleaned_docs)
#for doc in docs:
#    for word in doc:
#        if "happen" in word:
#            print("y")
            



In [77]:
#text_data=[]

#for doc in all_words:
#    for token in doc:
#        if random.random() > .60:
#            text_data.append(token)
#all_words = [text_data]

In [129]:
#np.random.seed(100)
dictionary = gensim.corpora.Dictionary(cleaned_docs)
dictionary.filter_extremes(no_below=1, no_above=0.5, keep_n=100000) # optional
bow_corpus = [dictionary.doc2bow(doc) for doc in cleaned_docs]
ldamodels = gensim.models.ldamodel.LdaModel(bow_corpus, num_topics = 4, id2word=dictionary, passes=30)

In [130]:
for i in ldamodels.print_topics(num_words = 18): 
    for j in i: print (j)

0
0.013*"layer" + 0.007*"create" + 0.007*"class" + 0.006*"bunch" + 0.006*"image" + 0.005*"convert" + 0.005*"numpy" + 0.005*"representation" + 0.005*"speak" + 0.005*"label" + 0.004*"earlier" + 0.004*"take" + 0.004*"loader" + 0.004*"test" + 0.004*"expect" + 0.004*"presentations" + 0.004*"docker" + 0.004*"issue"
1
0.013*"class" + 0.010*"figure" + 0.009*"match" + 0.009*"image" + 0.008*"decide" + 0.008*"think" + 0.007*"reward" + 0.006*"document" + 0.006*"context" + 0.006*"feature" + 0.006*"start" + 0.005*"francis" + 0.005*"talk" + 0.005*"accuracy" + 0.005*"notebook" + 0.005*"fix" + 0.005*"experiment" + 0.005*"decision"
2
0.012*"process" + 0.009*"record" + 0.007*"work" + 0.007*"elements" + 0.006*"change" + 0.006*"gallery" + 0.005*"emotion" + 0.005*"options" + 0.005*"talk" + 0.005*"choose" + 0.004*"question" + 0.004*"remove" + 0.004*"start" + 0.004*"dictionary" + 0.004*"write" + 0.004*"video" + 0.004*"think" + 0.004*"group"
3
0.011*"label" + 0.010*"color" + 0.009*"image" + 0.008*"platform" + 

In [131]:
# multicore model
#lda_model_multicore = gensim.models.LdaMulticore(bow_corpus, num_topics=4, id2word=dictionary, passes=2,workers=2)
#for idx, topic in lda_model_multicore.print_topics(-1):
#    print('Topic: {} \nWords: {}'.format(idx, topic))

In [132]:
#tdidf
#from gensim import corpora, models
#tfidf = models.TfidfModel(bow_corpus)
#corpus_tfidf = tfidf[bow_corpus]
#lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=4, id2word=dictionary, passes=2, workers=4)
#for idx, topic in lda_model_tfidf.print_topics(-1):
#    print('Topic: {} Word: {}'.format(idx, topic))

In [133]:
#from pprint import pprint
#for doc in corpus_tfidf:
#    pprint(doc)
#    break

## Visualising using pyLDAvis

In [134]:
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(ldamodels, bow_corpus, dictionary=ldamodels.id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2     -0.064590  0.111357       1        1  39.039425
3      0.127955 -0.008460       2        1  28.834433
1     -0.076922 -0.106876       3        1  25.103251
0      0.013557  0.003978       4        1   7.022900, topic_info=             Term       Freq      Total Category  logprob  loglift
41          class  22.000000  22.000000  Default  30.0000  30.0000
176         layer  11.000000  11.000000  Default  29.0000  29.0000
173         label  21.000000  21.000000  Default  28.0000  28.0000
456         color  17.000000  17.000000  Default  27.0000  27.0000
693       process  27.000000  27.000000  Default  26.0000  26.0000
..            ...        ...        ...      ...      ...      ...
184        listen   0.961474   6.076728   Topic4  -5.9700   0.8122
430         break   0.961410   3.898237   Topic4  -5.9701   1.2561
496        define   0.961405   4.703815   Topic4  -5.9701   1.0683
251  presentation   0.961396   5.997407   Topic4  -5.9701   0.8253
204       minutes   0.961379  12.448266   Topic4  -5.9701   0.0950

[273 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
0         1  0.311511  accuracy
0         3  0.726858  accuracy
385       1  0.136582    action
385       2  0.136582    action
385       3  0.682911    action
...     ...       ...       ...
381       2  0.151789     write
381       3  0.151789     write
853       1  0.772696     wrong
853       2  0.220770     wrong
853       4  0.110385     wrong

[442 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 4, 2, 1])

## Testing on unknown document

In [135]:
unseen_doc_file_path = './transcripts/test/autosampler-Onfido_2020-07-24_transcript.txt'
combined_words = ""
docs = []
data = open(unseen_doc_file_path).readlines()
speaker_data = {line.split(":")[0]:line.split(":")[1] for line in data}
words_in_file = ""
speaker_dic ={}
for name,words in  speaker_data.items():
    words = words.replace("\n","").lower()
    words_in_file = words_in_file + words
    if name.split("_")[0] in speaker_dic:
        speaker_dic[name.split("_")[0]] += words
    else:
        speaker_dic[name.split("_")[0]] = words
    #print("Number of words in the file :",str(len(words_in_file)))
combined_words += words_in_file
docs.append([words_in_file])
cleaned_docs = []
for doc in docs:
    for word in doc:
        cd = preprocess(word)
        cleaned_docs.append(cd)
        
bow_vector = dictionary.doc2bow(cleaned_docs[0])

for index, score in sorted(ldamodels[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, ldamodels.print_topic(index, 7)))       
    

Score: 0.41148248314857483	 Topic: 0.013*"class" + 0.010*"figure" + 0.009*"match" + 0.009*"image" + 0.008*"decide" + 0.008*"think" + 0.007*"reward"
Score: 0.24431341886520386	 Topic: 0.012*"process" + 0.009*"record" + 0.007*"work" + 0.007*"elements" + 0.006*"change" + 0.006*"gallery" + 0.005*"emotion"
Score: 0.20389311015605927	 Topic: 0.011*"label" + 0.010*"color" + 0.009*"image" + 0.008*"platform" + 0.007*"check" + 0.005*"docker" + 0.005*"fix"
Score: 0.14031095802783966	 Topic: 0.013*"layer" + 0.007*"create" + 0.007*"class" + 0.006*"bunch" + 0.006*"image" + 0.005*"convert" + 0.005*"numpy"


In [136]:
res = ldamodels[bow_vector]

In [137]:
print (res)

[(0, 0.1403192), (1, 0.41148248), (2, 0.24431631), (3, 0.203882)]


 it is suggesting topic 0